# Machine Learning with Text

blah

dataset: https://www.kaggle.com/utathya/imdb-review-dataset
source: http://www.cs.cornell.edu/people/pabo/movie-review-data/

In [ ]:
#!kaggle datasets download -d utathya/imdb-review-dataset
#!kaggle datasets download -d oumaimahourrane/imdb-movie-reviews-cleaned-data

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy import sparse
from xgboost import XGBClassifier
import shap
from pathlib import Path
import os
import spacy
from wordcloud import WordCloud
np.random.seed(42)

import warnings
warnings.filterwarnings('ignore')

In [55]:
def get_text(text_file):
    df = pd.read_csv(text_file, encoding='ISO-8859-1', index_col=0)
    df = df[['label','review']]
    df = df[df.label != 'unsup']
    df.label = df.label.replace('neg',0)
    df.label = df.label.replace('pos',1)
    df = df.sample(frac=1).reset_index(drop=True)
    return df

def clean_text(text):
    pass

def show_wordcloud(text):
    text = ''.join([t for t in text])
    wordcloud = WordCloud().generate(text)
    plt.figure(figsize=(8,8))
    plt.imshow(wordcloud)
    plt.axes('off')
    plt.show()

In [56]:
text_file = Path('~/.kaggle/datasets/utathya/imdb-review-dataset/imdb_master.csv')
#text_file = Path('~/.kaggle/datasets/oumaimahourrane/imdb-movie-reviews-cleaned-data/clean_data.csv')
df = get_text(text_file)
print(df.info())
print()
print(df.label.value_counts())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
label     50000 non-null int64
review    50000 non-null object
dtypes: int64(1), object(1)
memory usage: 781.3+ KB
None

1    25000
0    25000
Name: label, dtype: int64


,label,review
0,0,Imagine The Big Chill with a cast of twenty-so...
1,0,I'd have to say that I've seen worse Sci Fi Ch...
2,0,Director Fabio Barreto got a strange Academy N...
3,0,Pretty bad PRC cheapie which I rarely bother t...
4,1,This is a very intriguing short movie by David...


In [57]:
review = df.iloc[0]
print('label:',review.label)
print(review.review)

print()

review = df.iloc[5]
print('label:',review.label)
print(review.review)

label: 0
Imagine The Big Chill with a cast of twenty-somethings whose characters are all unlikable, and an iguana-like man-lizard chasing them around and you have an idea of the foolishness herein. On the positive side, the movie does not skimp on showing the monster. There's no peek-a-boo shots, or nighttime scenes where you have to imagine what he looks like; he's right out there folks. Unfortunately, the design and construction aren't that inspired. A little bit of mystery might have helped. Mind you, I've seen far worse, but if you're going to have him out on full display for a lot of the flick, your monster better look damn good. <br /><br />Spoiler Ahead!!!<br /><br />Oh, and there's a twist ending involving the supposedly dead brother that makes absolutely no sense whatsoever. This came as no surprise given the shoddy writing of the script. As for the acting; well let's just say it wasn't painfully bad, but I don't expect we'll be seeing many of these kids in future cinematic ou

In [58]:
review = df.iloc[0]
doc = review.review.split(' ')
[token for token in doc]

['Imagine',
 'The',
 'Big',
 'Chill',
 'with',
 'a',
 'cast',
 'of',
 'twenty-somethings',
 'whose',
 'characters',
 'are',
 'all',
 'unlikable,',
 'and',
 'an',
 'iguana-like',
 'man-lizard',
 'chasing',
 'them',
 'around',
 'and',
 'you',
 'have',
 'an',
 'idea',
 'of',
 'the',
 'foolishness',
 'herein.',
 'On',
 'the',
 'positive',
 'side,',
 'the',
 'movie',
 'does',
 'not',
 'skimp',
 'on',
 'showing',
 'the',
 'monster.',
 "There's",
 'no',
 'peek-a-boo',
 'shots,',
 'or',
 'nighttime',
 'scenes',
 'where',
 'you',
 'have',
 'to',
 'imagine',
 'what',
 'he',
 'looks',
 'like;',
 "he's",
 'right',
 'out',
 'there',
 'folks.',
 'Unfortunately,',
 'the',
 'design',
 'and',
 'construction',
 "aren't",
 'that',
 'inspired.',
 'A',
 'little',
 'bit',
 'of',
 'mystery',
 'might',
 'have',
 'helped.',
 'Mind',
 'you,',
 "I've",
 'seen',
 'far',
 'worse,',
 'but',
 'if',
 "you're",
 'going',
 'to',
 'have',
 'him',
 'out',
 'on',
 'full',
 'display',
 'for',
 'a',
 'lot',
 'of',
 'the',
 'flick,',
 'your',
 'monster',
 'better',
 'look',
 'damn',
 'good.',
 '<br',
 '/><br',
 '/>Spoiler',
 'Ahead!!!<br',
 '/><br',
 '/>Oh,',
 'and',
 "there's",
 'a',
 'twist',
 'ending',
 'involving',
 'the',
 'supposedly',
 'dead',
 'brother',
 'that',
 'makes',
 'absolutely',
 'no',
 'sense',
 'whatsoever.',
 'This',
 'came',
 'as',
 'no',
 'surprise',
 'given',
 'the',
 'shoddy',
 'writing',
 'of',
 'the',
 'script.',
 'As',
 'for',
 'the',
 'acting;',
 'well',
 "let's",
 'just',
 'say',
 'it',
 "wasn't",
 'painfully',
 'bad,',
 'but',
 'I',
 "don't",
 'expect',
 "we'll",
 'be',
 'seeing',
 'many',
 'of',
 'these',
 'kids',
 'in',
 'future',
 'cinematic',
 'outings.<br',
 '/><br',
 '/>Gore',
 'quotient:',
 '2',
 'out',
 'of',
 '5;',
 'Nudity',
 'quotient:',
 '1',
 'out',
 'of',
 '5;',
 'Intelligence',
 'quotient:',
 'Negligible']

In [59]:
nlp = spacy.load('en_core_web_sm')
review = df.iloc[0]
doc = nlp(review.review)
[token.text for token in doc]

['Imagine',
 'The',
 'Big',
 'Chill',
 'with',
 'a',
 'cast',
 'of',
 'twenty',
 '-',
 'somethings',
 'whose',
 'characters',
 'are',
 'all',
 'unlikable',
 ',',
 'and',
 'an',
 'iguana',
 '-',
 'like',
 'man',
 '-',
 'lizard',
 'chasing',
 'them',
 'around',
 'and',
 'you',
 'have',
 'an',
 'idea',
 'of',
 'the',
 'foolishness',
 'herein',
 '.',
 'On',
 'the',
 'positive',
 'side',
 ',',
 'the',
 'movie',
 'does',
 'not',
 'skimp',
 'on',
 'showing',
 'the',
 'monster',
 '.',
 'There',
 "'s",
 'no',
 'peek',
 '-',
 'a',
 '-',
 'boo',
 'shots',
 ',',
 'or',
 'nighttime',
 'scenes',
 'where',
 'you',
 'have',
 'to',
 'imagine',
 'what',
 'he',
 'looks',
 'like',
 ';',
 'he',
 "'s",
 'right',
 'out',
 'there',
 'folks',
 '.',
 'Unfortunately',
 ',',
 'the',
 'design',
 'and',
 'construction',
 'are',
 "n't",
 'that',
 'inspired',
 '.',
 'A',
 'little',
 'bit',
 'of',
 'mystery',
 'might',
 'have',
 'helped',
 '.',
 'Mind',
 'you',
 ',',
 'I',
 "'ve",
 'seen',
 'far',
 'worse',
 ',',
 'but',
 'if',
 'you',
 "'re",
 'going',
 'to',
 'have',
 'him',
 'out',
 'on',
 'full',
 'display',
 'for',
 'a',
 'lot',
 'of',
 'the',
 'flick',
 ',',
 'your',
 'monster',
 'better',
 'look',
 'damn',
 'good',
 '.',
 '<',
 'br',
 '/><br',
 '/>Spoiler',
 'Ahead!!!<br',
 '/><br',
 '/>Oh',
 ',',
 'and',
 'there',
 "'s",
 'a',
 'twist',
 'ending',
 'involving',
 'the',
 'supposedly',
 'dead',
 'brother',
 'that',
 'makes',
 'absolutely',
 'no',
 'sense',
 'whatsoever',
 '.',
 'This',
 'came',
 'as',
 'no',
 'surprise',
 'given',
 'the',
 'shoddy',
 'writing',
 'of',
 'the',
 'script',
 '.',
 'As',
 'for',
 'the',
 'acting',
 ';',
 'well',
 'let',
 "'s",
 'just',
 'say',
 'it',
 'was',
 "n't",
 'painfully',
 'bad',
 ',',
 'but',
 'I',
 'do',
 "n't",
 'expect',
 'we',
 "'ll",
 'be',
 'seeing',
 'many',
 'of',
 'these',
 'kids',
 'in',
 'future',
 'cinematic',
 'outings.<br',
 '/><br',
 '/>Gore',
 'quotient',
 ':',
 '2',
 'out',
 'of',
 '5',
 ';',
 'Nudity',
 'quotient',
 ':',
 '1',
 'out',
 'of',
 '5',
 ';',
 'Intelligence',
 'quotient',
 ':',
 'Negligible']

In [30]:
doc

Imagine The Big Chill with a cast of twenty-somethings whose characters are all unlikable, and an iguana-like man-lizard chasing them around and you have an idea of the foolishness herein. On the positive side, the movie does not skimp on showing the monster. There's no peek-a-boo shots, or nighttime scenes where you have to imagine what he looks like; he's right out there folks. Unfortunately, the design and construction aren't that inspired. A little bit of mystery might have helped. Mind you, I've seen far worse, but if you're going to have him out on full display for a lot of the flick, your monster better look damn good. <br /><br />Spoiler Ahead!!!<br /><br />Oh, and there's a twist ending involving the supposedly dead brother that makes absolutely no sense whatsoever. This came as no surprise given the shoddy writing of the script. As for the acting; well let's just say it wasn't painfully bad, but I don't expect we'll be seeing many of these kids in future cinematic outings.<br

In [60]:
cv = CountVectorizer(max_features=60000)
X_counts = cv.fit_transform(df.review)
y = df.label

print(X_counts.shape,y.shape)

(50000, 60000) (50000,)


In [6]:
tfidf = TfidfTransformer()
X_tfidf = tfidf.fit_transform(X_counts)

print(X_tfidf.shape)

(50000, 60000)


In [7]:
X_train,X_test,y_train,y_test = train_test_split(X_tfidf, y, test_size=0.2) 

In [11]:
model = MultinomialNB()
model.fit(X_train,y_train)
acc = model.score(X_test,y_test)
print(acc)

0.8637


In [8]:
model = RandomForestClassifier(n_estimators=50,n_jobs=-1)
model.fit(X_train,y_train)
acc = model.score(X_test,y_test)
print(acc)

0.8255


In [9]:
warnings.filterwarnings('ignore')
os.environ['KMP_DUPLICATE_LIB_OK']='True'

model = XGBClassifier(n_estimators=50,n_jobs=-1)
model.fit(X_train,y_train)
acc = model.score(X_test,y_test)
print(acc)

0.7758
